In [1]:
%load_ext catalyst

In [2]:
# Setup matplotlib to display graphs inline in this Notebook
%matplotlib inline

In [77]:
#Distance approach (by Bogomolov)

from datetime import timedelta

import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt


from catalyst import run_algorithm
from catalyst.api import (symbols, order, symbol, order_target_percent, order_percent, record)
from catalyst.exchange.utils.exchange_utils import get_exchange_symbols


def initialize(context):
    context.i = -1  # minute/daily bar counter
    context.exchange = list(context.exchanges.values())[0].name.lower() #bitfinex
    context.quote_currency = list(
        context.exchanges.values())[0].quote_currency.lower()
    #STRATEGY
    # We need to make a matrix of UNIVERSE CLOSE prices for a given FORMATION_PERIOD and TRADING_PERIOD
    context.price = 'close' # we use 'close' to make computations 
    context.formation_period = 200 #in bars
    context.trading_period = 40 # in bars
#     context.number_of_trading_periods = 100 #number of periods we want to trade
    context.number_of_coins = 10 #number of TOP coins by VOLUME in universe we want to trade
    context.number_of_pairs = 5 #number of coin pairs we want to form
    context.minutes = 60  #data resolution 
    context.frequency = '{}T'.format(context.minutes) #frequency for data.history()
    context.tp_count = -1 #trading period counter
    context.pairs_universe = None #universe of trading pairs
    context.std_open_mult = 2 #number of standard deviations to open position
    context.std_sl_mult = 3 #number of standard deviations to stop-loss position
    context.std_close_mult = 0.5 #number of std to close position (take profit)
    # NEED TO EXTEND NUMBER OF COINS
    context.explicit_coins = ['btc_usd', 'eth_usd', 'xrp_usd', 'ltc_usd', 'eos_usd', 'iot_usd', 'neo_usd', 'etc_usd', 'zec_usd', 'dsh_usd', 'xmr_usd', 'trx_usd']
    context.set_commission(maker=0.00, taker=0.00) # we can play with comission to look how profits changes

def handle_data(context, data):
    #minute counter
    context.i += 1
    
    if data.current_dt <= pd.to_datetime('2018-05-01', utc=True):
        return
    #FORMATION LOGIC
    #We skip first FP bars
#     if context.i < context.formation_period*context.minutes:
#         return
    #Every beginning of TP we look FP bars ago and get universe of existing assets for this period.
    if context.i % (context.trading_period*context.minutes) == 0:
        now = data.current_dt
        past = now - timedelta(minutes=context.formation_period*context.minutes)
        #Next is commented since there is missing data in some periods. We use context.explicit_coins instead below...
#         context.universe = universe_pt(context, now, past)  #form universe (list of assets to upload in history)
        context.coins = symbols(*context.explicit_coins)  # convert all the pairs to SYMBOLS format which is necessary for data.history
        
        #To simplify computation, we want to reduce our universe of coins according to rule of highest volume in a day
        yesterday_volume = data.history(context.coins,'volume',bar_count=2,frequency='1D')[0:1].T #we get volume in BASE currency
        yesterday_close = data.history(context.coins,'close',bar_count=2,frequency='1D')[0:1].T
        yesterday_volume = yesterday_volume.nlargest(context.number_of_coins, columns=yesterday_volume.columns[0]) # we pick only high traded coins
        context.coins_reduced = list(yesterday_volume.index) # the same as making by symbols function
        #close positions if they are open
        for coin in context.portfolio.positions:
            order_target_percent(coin, 0)
        display(context.portfolio.portfolio_value, data.current_dt)

    if context.i % (context.trading_period*context.minutes) == 0:  
        #1)we take prices
        close = data.history(context.coins_reduced,'close',bar_count=context.formation_period,frequency=context.frequency)
        #2)then we normalize them
        close_log = close.apply(np.log)
        #3)then we make all possible spreads
        context.pairs_spread = pd.DataFrame(index=close_log.index)
        context.pairs_spread_std = pd.DataFrame(index=['std']) # dm is for distance measure
        #Here we make combinations of all possible spread using itertools
        for i,j in itertools.combinations(close_log.columns,2):           
            context.pairs_spread[(i,j)] = close_log[i] - close_log[j] 
            context.pairs_spread[(i,j)] = context.pairs_spread[(i,j)] - context.pairs_spread[(i,j)].mean()
            context.pairs_spread_std[(i,j)] = context.pairs_spread[(i,j)].std()
        context.full_universe = context.pairs_spread_std.columns
        # 5) now we reduce universe to top NUMBER_OF_PAIRS pairs according by lowest distance measure:
        context.pairs_universe = list(context.pairs_spread_std.sort_values(by='std', axis=1).columns[: context.number_of_pairs])
#         display(context.i, pairs_spread_std, pairs_spread, pairs_spread_std.sort_values(by='std', axis=1))
        # 6) we create empty dataset for future trading data
        context.dataset = pd.DataFrame(index=range(context.trading_period), columns=context.pairs_universe) 
        context.full_dataset = pd.DataFrame(index=range(context.trading_period), columns=context.full_universe)
        # 7) and compute all standard deviations for trading logic
        for pair in context.pairs_universe:
            context.dataset.loc['std', pair] = context.pairs_spread[pair].std()
            context.dataset.loc['mean', pair] = (close_log[pair[0]] - close_log[pair[1]]).mean()
            context.full_dataset.loc['std', pair] = context.pairs_spread[pair].std()

#             plt.figure()
#             plt.plot(pairs_spread[pair])
#             plt.axhline(y=context.dataset[pair].loc['std'], color='r', linestyle='-')
#             plt.show()
        #for positions: long or short
        context.pos = pd.DataFrame(index = range(context.trading_period), columns=context.pairs_universe)
        context.pos.iloc[0] = 0
        #for bans of long or short
        context.pos_ban = pd.DataFrame(index = range(context.trading_period), columns=context.pairs_universe)
        #for orders logic. we get id of open transactions to close it further.
        context.transactions_open = pd.DataFrame(index=(0,1), columns=context.pairs_universe)


        
    #TRADING LOGIC
    
    if context.i % context.minutes == 0 and context.pairs_universe: #we trade each CONTEXT.MINUTES minutes and need pairs_universe to exist
        context.tp_count = int(context.i%(context.trading_period*context.minutes)/context.minutes) # goes from 0 to (context.trading_period - 1)
#         display(context.tp_count)
        for pair in context.dataset.columns: # here pair contains TWO coins
            current_price = data.current(pair, context.price)
            current_price_log = current_price.apply(np.log)
            context.dataset.loc[context.tp_count, pair] = current_price_log[0] - current_price_log[1] - context.dataset[pair].loc['mean'] # here we get pair log-spread 
#             display(context.std_open_mult*context.dataset[pair].loc[context.tp_count], 'std: ', context.dataset[pair].loc['std'])
#             context.full_dataset.loc[context.tp_count, pair] = context.dataset[pair].loc[context.tp_count]
# Check if it's not 0 bar of trading (it is the bar when we form our pairs_universe)
            if context.tp_count>0:
                #we assign current positions and bans as previous ones
                context.pos.loc[context.tp_count, [pair]] = context.pos.loc[context.tp_count-1, [pair]]
                context.pos_ban.loc[context.tp_count, [pair]] = context.pos_ban.loc[context.tp_count-1, [pair]]
                # then we start to manipulate with current positions if they comply with some conditions
                #if BAN
                if type(context.pos_ban[pair].loc[context.tp_count])==str:
                    if (context.pos_ban[pair].loc[context.tp_count]=='short' and 
                           context.dataset[pair].loc[context.tp_count] < context.std_open_mult*context.dataset[pair].loc['std']):
                        context.pos_ban.loc[context.tp_count, pair] = np.nan
#                         print(context.i, 'unban of short: {}, {}'.format(pair[0], pair[1]))
                    elif (context.pos_ban[pair].loc[context.tp_count]=='long' and 
                           context.dataset[pair].loc[context.tp_count] > -context.std_open_mult*context.dataset[pair].loc['std']):
                        context.pos_ban.loc[context.tp_count, pair] = np.nan
#                         print(context.i, 'unban of long: {}, {}'.format(pair[0], pair[1]))
                
                #If NO POSITION:
                if context.pos[pair].loc[context.tp_count] == 0 and pd.isnull(context.pos_ban[pair].loc[context.tp_count]):
                    #SHORT SPREAD
                    if (context.dataset[pair].loc[context.tp_count] > context.std_open_mult*context.dataset[pair].loc['std']):
#                         print(context.i, 'short {} long {}'.format(pair[0], pair[1]))
                        context.pos.loc[context.tp_count, pair] = -1
                    #LONG SPREAD    
                    elif (context.dataset[pair].loc[context.tp_count] < -context.std_open_mult*context.dataset[pair].loc['std']):
#                         print(context.i, 'long {} short {}'.format(pair[0], pair[1]))
                        context.pos.loc[context.tp_count, pair] = 1
                #If SHORT POSITION:
                if context.pos[pair].loc[context.tp_count] == -1:
                    #TAKE PROFIT SHORT
                    if context.dataset[pair].loc[context.tp_count] < context.std_close_mult*context.dataset[pair].loc['std']:
#                         print(context.i, 'take profit of short: {}, {}'.format(pair[0], pair[1])) 
                        context.pos.loc[context.tp_count, pair] = 0
                    #STOP-LOSS SHORT
                    elif context.dataset[pair].loc[context.tp_count] > context.std_sl_mult*context.dataset[pair].loc['std']:
#                         print(context.i, 'stop-loss of short: {}, {}'.format(pair[0], pair[1])) 
                        context.pos.loc[context.tp_count, pair] = 0
                        context.pos_ban.loc[context.tp_count,pair] = 'short'
                #If LONG POSITION:
                if context.pos[pair].loc[context.tp_count] == 1:
                    #TAKE PROFIT LONG
                    if context.dataset[pair].loc[context.tp_count] > -context.std_close_mult*context.dataset[pair].loc['std']:
#                         print(context.i, 'take profit of long: {}, {}'.format(pair[0], pair[1])) 
                        context.pos.loc[context.tp_count,pair] = 0
                    #STOP-LOSS LONG
                    elif context.dataset[pair].loc[context.tp_count] < -context.std_sl_mult*context.dataset[pair].loc['std']:
#                         print(context.i, 'stop-loss of long: {}, {}'.format(pair[0], pair[1])) 
                        context.pos.loc[context.tp_count, pair] = 0
                        context.pos_ban.loc[context.tp_count, pair] = 'long'
#             print(context.i,'\nFirst pair is:', pair[0], '\nSecond pair is:', pair[1])
#         display(context.tp_count, context.i, current_price, current_price_log, context.dataset)
        record(spread_dataset = context.full_dataset.loc[context.tp_count,:],
          std_dataset = context.full_dataset.loc['std',:],
              pos_dataset = context.pos.loc[context.tp_count,:],
                  pos_ban_dataset = context.pos_ban.loc[context.tp_count,:],
                      btc=data.current(symbol('btc_usd'), 'price'))
#         display(context.tp_count, context.i, context.pos, context.pos_ban, context.dataset)

    #ORDER LOGIC
    if (context.tp_count>0 and 
            not context.pos.loc[context.tp_count].equals(context.pos.loc[context.tp_count-1]) and 
                context.i % context.minutes == 0):
        #difference between current postitions and previous
        difference = context.pos.loc[context.tp_count] - context.pos.loc[context.tp_count-1]
        #we leave only changed positions
        difference = difference[difference!=0]
        for pair in difference.index:
            #if we don't have to close previously open positions
            if pd.isnull(context.transactions_open[pair].loc[0]):
                # we write id of each opening position
                context.transactions_open.loc[0,pair]  = order_percent(pair[0], (1/context.number_of_pairs)*difference[pair])
                context.transactions_open.loc[1,pair]  = order_percent(pair[1], -(1/context.number_of_pairs)*difference[pair])
            else:
                # we want to close opened positions
                order(pair[0], -float(context.blotter.orders[context.transactions_open[pair].loc[0]].amount))
                order(pair[1], -float(context.blotter.orders[context.transactions_open[pair].loc[1]].amount))
                context.transactions_open[pair] = np.nan
                
#             print(context.transactions_open[pair])
#         display(context.i, data.current_dt, context.portfolio.positions)

        # end trading at the end of TP:        
    if context.tp_count == context.trading_period-1 and (context.i % context.minutes) - 1 == 0 and context.pairs_universe:
        context.pairs_universe = None
#         print(context.i, 'End of trading period')
#         display(context.tp_count, context.i, context.pos, context.pos_ban, context.dataset)
#         display(context.blotter.orders)
    
def analyze(context=None, results=None):
    pass


# Function is working but corrupts when there are missing data field in inngested budles. 
def universe_pt(context, now, past):
    # get all the pairs for the given exchange
    json_symbols = get_exchange_symbols(context.exchange)
    df_universe = pd.DataFrame.from_dict(json_symbols).transpose()
#     display(df_universe[df_universe['symbol'] == context.quote_currency])
    df_universe['quote_currency'] = df_universe.apply(lambda row: row.symbol.split('_')[1],
                                    axis=1)
    df_universe['base_currency'] = df_universe.apply(lambda row: row.symbol.split('_')[0],
                                   axis=1)
    # Filter all the pairs to get only the ones for a given quote_currency
    df_universe = df_universe[df_universe['quote_currency'] == context.quote_currency]
    # Check if type of each row is pd.tslib.Timestamp, drop otherwise.
    df_universe = df_universe[df_universe.end_minute.apply(isinstance, args=(pd.tslib.Timestamp,))]
    df_universe = df_universe[df_universe.start_date.apply(isinstance, args=(pd.tslib.Timestamp,))]
    df_universe = df_universe[df_universe.end_minute > now]
    df_universe = df_universe[df_universe.start_date < past]
    return df_universe.symbol.tolist()

# Replace all NA, NAN or infinite values with its nearest value. NOT USED NOW.
def fill(series):
    if isinstance(series, pd.Series):
        return series.replace([np.inf, -np.inf], np.nan).ffill().bfill()
    elif isinstance(series, np.ndarray):
        return pd.Series(series).replace(
                     [np.inf, -np.inf], np.nan
                    ).ffill().bfill().values
    else:
        return series


if __name__ == '__main__':
    # NEED TO EXTEND SIMULATION PERIOD to at least 2017 year. In minute and daily data resolution. 
    # SOMETIMES THERE ARE MISSING DATA IN CATALYST DATABASE, SO BE CAREFUL WHEN INGESTING DATA.
    start_date = pd.to_datetime('2018-04-20', utc=True)
    end_date = pd.to_datetime('2019-05-01', utc=True)

    performance = run_algorithm(start=start_date, end=end_date,
                                capital_base=100.0,  # amount of quote_currency to trade
                                initialize=initialize,
                                handle_data=handle_data,
                                analyze=analyze,
                                exchange_name='bitfinex',
                                data_frequency='minute', # also daily is possible
                                quote_currency='usd',
                                live=False,
                                live_graph=False,
                                algo_namespace='simple_universe')

[2019-06-10 16:26:55.148118] INFO: run_algo: Catalyst version 0.5.21
[2019-06-10 16:26:55.161302] WARNING: run_algo: Catalyst is currently in ALPHA. It is going through rapid development and it is subject to errors. Please use carefully. We encourage you to report any issue on GitHub: https://github.com/enigmampc/catalyst/issues
[2019-06-10 16:26:58.165453] INFO: run_algo: running algo in backtest mode
[2019-06-10 16:26:58.234361] INFO: exchange_algorithm: initialized trading algorithm in backtest mode


100.0

Timestamp('2018-05-01 16:00:00+0000', tz='UTC')

99.93149991016547

Timestamp('2018-05-03 08:00:00+0000', tz='UTC')

...

105.12452992575246

Timestamp('2019-04-30 00:00:00+0000', tz='UTC')

107.06016668432353

Timestamp('2019-05-01 16:00:00+0000', tz='UTC')

[2019-06-10 17:00:17.406388] INFO: Performance: Simulated 377 trading days out of 377.
[2019-06-10 17:00:17.428981] INFO: Performance: first open: 2018-04-20 00:00:00+00:00
[2019-06-10 17:00:17.430398] INFO: Performance: last close: 2019-05-01 23:59:00+00:00


In [78]:
# here is how to call performance after simulation of data
display(performance)
#this is how to save it for later use
performance.to_pickle('/Users/vladimirbadlo/StatArb/Results/perf_5.18_5.19_B_fp200_tp40_np5_60_2_3_0.5')

In [ ]:
#For testing smth
from catalyst.api import (symbols, order, symbol, record, order_target)
import numpy as np
import pandas as pd

def initialize(context):
    context.asset_btc = symbol('btc_usdt')
    context.asset_eth = symbol('eth_usdt')
    context.exchange = 'bitfinex'
    context.i = -1


def handle_data(context, data):
    context.i += 1
    if context.i  == 0:
        order(context.asset_btc, -1)
        order_target(context.asset_btc, -3)
        #order(context.asset_eth, 10)
        
    record(btc = data.current(context.asset_btc, 'price'))

def analyze(context, perf):
    pass


if __name__ == '__main__':
    start_date = pd.to_datetime('2019-03-01', utc=True)
    end_date = pd.to_datetime('2019-03-05', utc=True)

    performance = run_algorithm(start=start_date, end=end_date,
                                capital_base=100.0,  # amount of quote_currency
                                initialize=initialize,
                                handle_data=handle_data,
                                analyze=analyze,
                                exchange_name='bitfinex',
                                data_frequency='daily',
                                quote_currency='usd',
                                live=False,
                                live_graph=False,
                                algo_namespace='simple_universe')

In [ ]:
# current universe of coins. You can supply it in 'catalyst ingest-exchange' function in terminal
btc_usd,eth_usd,xrp_usd,ltc_usd,eos_usd,iot_usd,neo_usd,etc_usd,zec_usd,dsh_usd,xmr_usd,xlm_usd,trx_usd